# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

In [1]:
import findspark
findspark.init('/home/macaubas/spark-3.2.1-bin-hadoop3.2')
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tree').getOrCreate()

22/05/19 15:55:12 WARN Utils: Your hostname, macaubas-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/05/19 15:55:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/19 15:55:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/19 15:55:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Imports

In [2]:
from pyspark.ml import Pipeline

from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier, DecisionTreeClassifier)

from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

from pyspark.ml.feature import (VectorAssembler, StringIndexer)

## Carregando dados

In [3]:
data = spark.read.csv('dog_food.csv', inferSchema=True, header=True)

data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [4]:
for e in data.head(5):
    print(e)

Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)
Row(A=5, B=6, C=12.0, D=7, Spoiled=1.0)
Row(A=6, B=2, C=13.0, D=6, Spoiled=1.0)
Row(A=4, B=2, C=12.0, D=1, Spoiled=1.0)
Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)


## Assembler

In [5]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [6]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],
                           outputCol='features')

In [7]:
output = assembler.transform(data)

In [8]:
final_data = output.select('features', 'Spoiled')

final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Spoiled: double (nullable = true)



In [10]:
for e in final_data.head(5):
    print(e)
    print('\n')

Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)


Row(features=DenseVector([5.0, 6.0, 12.0, 7.0]), Spoiled=1.0)


Row(features=DenseVector([6.0, 2.0, 13.0, 6.0]), Spoiled=1.0)


Row(features=DenseVector([4.0, 2.0, 12.0, 1.0]), Spoiled=1.0)


Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)




## Modelo

In [11]:
rfc_1 = RandomForestClassifier(labelCol='Spoiled', featuresCol='features', numTrees=25)
rfc_2 = RandomForestClassifier(labelCol='Spoiled', featuresCol='features', numTrees=100)
rfc_3 = RandomForestClassifier(labelCol='Spoiled', featuresCol='features', numTrees=150)

In [13]:
rfc_1_model = rfc_1.fit(final_data)
rfc_2_model = rfc_2.fit(final_data)
rfc_3_model = rfc_3.fit(final_data)

## Avaliando qual fator que mais contribui com spoiled

In [14]:
## Modelo 1
rfc_1_model.featureImportances

SparseVector(4, {0: 0.0243, 1: 0.0277, 2: 0.9262, 3: 0.0218})

In [15]:
## Modelo 2
rfc_2_model.featureImportances

SparseVector(4, {0: 0.0185, 1: 0.0211, 2: 0.9406, 3: 0.0198})

In [16]:
## Modelo 3
rfc_3_model.featureImportances

SparseVector(4, {0: 0.0217, 1: 0.0207, 2: 0.936, 3: 0.0216})


O produto **químico C** é o que mais está contribuindo no perecimento das rações dos cachorros!
-----

# Good Luck!